## Calibration tests
### Pose2Sim calibration.py functions
### Data projet Cobotique

Auteurs : Léa Drolet-Roy

Création : 2025-02-18

Dernière modification :

In [1]:
from pose2sim_trampo import calibration
import os
import time
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from scipy.spatial.transform import Rotation as R
from scipy.spatial import procrustes
from scipy.linalg import orthogonal_procrustes
from scipy.sparse import lil_matrix
from scipy.optimize import least_squares

import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
from pyswarms.utils.plotters import (plot_cost_history, plot_contour, plot_surface)

from Calibration import calibration as cali

np.set_printoptions(precision=4, suppress=True)

### Constants

In [2]:
CHECKERBOARD = (7,4)
SQUARE_SIZE = 48

cams = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6']
time_threshold = 30

path = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique'
u101 = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\u101'
u102 = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\u102'

extrinsics_corners_nb = CHECKERBOARD
extrinsics_square_size = SQUARE_SIZE

objpoints = np.zeros((extrinsics_corners_nb[0] * extrinsics_corners_nb[1], 3), np.float32)
objpoints[:, :2] = np.mgrid[0:extrinsics_corners_nb[0], 0:extrinsics_corners_nb[1]].T.reshape(-1, 2)
objpoints[:, :2] = objpoints[:, 0:2] * extrinsics_square_size

cal = cali(CHECKERBOARD, SQUARE_SIZE)

In [3]:
# for cam in os.listdir(u101):
#     cal.save_images_checkerboard(path, u101, cam)
    
# for cam in os.listdir(u102):
#    cal.save_images_checkerboard(path, u102, cam)

In [33]:
intrinsics_config_dict = {'calculate_intrinsics':True, 'overwrite_intrinsics':True, 'show_detection_intrinsics':True, 'intrinsics_method':'board',
                          'intrinsics_extension': 'png', 'intrinsics_corners_nb':CHECKERBOARD, 'intrinsics_square_size':SQUARE_SIZE,
                          'board': {'intrinsics_extension': 'png', 'show_reprojection_error': True,
                                    'intrinsics_corners_nb': CHECKERBOARD, 'intrinsics_square_size':SQUARE_SIZE}}

extrinsics_config_dict = {'calculate_extrinsics':True, 'show_detection_extrinsics':False, 'extrinsics_method':'board',
                          'extrinsics_extension': 'png', 'extrinsics_corners_nb':CHECKERBOARD, 'extrinsics_square_size':SQUARE_SIZE,
                          'board': {'extrinsics_extension': 'png', 'show_reprojection_error': True,
                                    'extrinsics_corners_nb': CHECKERBOARD, 'extrinsics_square_size':SQUARE_SIZE}}

calib_dir = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration"

image_size = (1280, 720)
camera_matrix = np.array([[644, 0, 656],
                         [0, 643, 358],
                         [0,0,1]], dtype=np.float32)

ret, C, S, D, K, R, T = calibration.calibrate_intrinsics(calib_dir, intrinsics_config_dict)

# ret, C, S, D, K, R, T = cal.calibrate_intrinsics(calib_dir, image_size, camera_matrix)

# print(f'reproj error (mm): {ret} \nCam: {C} \nImage Size: {S} \nIntrinsics: \n{K[0]} \n{K[1]} \n{K[2]} \n{K[3]} \n{K[4]} \n{K[5]}')



2025-03-05 10:49:26,143 - root - INFO - 
Camera c1:
2025-03-05 10:49:27,584 - root - INFO - g1_p3_0.png: Corners found.
2025-03-05 10:49:31,550 - root - INFO - g1_p3_1.png: Corners found.
2025-03-05 10:49:33,468 - root - INFO - g1_p3_10.png: Corners found.
2025-03-05 10:49:35,047 - root - INFO - g1_p3_11.png: Corners found.
2025-03-05 10:49:37,491 - root - INFO - g1_p3_12.png: Corners found.
2025-03-05 10:49:38,788 - root - INFO - g1_p3_13.png: Corners found.
2025-03-05 10:49:39,984 - root - INFO - g1_p3_14.png: Corners found.
2025-03-05 10:49:41,384 - root - INFO - g1_p3_15.png: Corners found.
2025-03-05 10:49:42,555 - root - INFO - g1_p3_16.png: Corners found.
2025-03-05 10:49:43,512 - root - INFO - g1_p3_17.png: Corners found.
2025-03-05 10:49:44,635 - root - INFO - g1_p3_18.png: Corners found.
2025-03-05 10:49:45,744 - root - INFO - g1_p3_19.png: Corners found.
2025-03-05 10:49:46,844 - root - INFO - g1_p3_27.png: Corners found.
2025-03-05 10:49:53,544 - root - INFO - g1_p3_28.png:

In [34]:
K = np.array(K)
np.savez('Intrinsics_K_manual.npz', K)

D = np.array(D)
np.savez('Intrinsics_D_manual.npz', D)


In [4]:
K = np.load('Intrinsics_K.npz')['arr_0']
D = np.load('Intrinsics_D.npz')['arr_0']


In [5]:
angle_threshold = 60 #degrees
coverage_threshold = (0.01, 0.95)
image_size = (1280, 720)

for i, cam in enumerate(cams):
    area = cal.refine_image_selection(calib_dir, 'intrinsics_refine', cam, K[i], D[i], image_size, angle_threshold, coverage_threshold)
    print(area)

0.5435785590277777
0.8144401041666667
0.8598415798611111
0.7568663194444445
0.7525249565972222
0.7624370659722223


In [8]:
calib_dir = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration\intrinsics_refine"

ret, C, S, D, K, R, T = cal.calibrate_intrinsics(calib_dir, image_size, K, D)

print(f'reproj error (mm): {ret} \nCam: {C} \nImage Size: {S} \nIntrinsics: \n{K[0]} \n{K[1]} \n{K[2]} \n{K[3]} \n{K[4]} \n{K[5]}')


reproj error (mm): [0.23570284893008173, 0.26436340583937495, 0.367739615244798, 0.47195161181436684, 0.8721917186769076, 0.3754290849362984] 
Cam: ['c1', 'c2', 'c3', 'c4', 'c5', 'c6'] 
Image Size: [[1280.0, 720.0], [1280.0, 720.0], [1280.0, 720.0], [1280.0, 720.0], [1280.0, 720.0], [1280.0, 720.0]] 
Intrinsics: 
[[647.3789   0.     647.8307]
 [  0.     649.6502 371.5707]
 [  0.       0.       1.    ]] 
[[659.0669   0.     656.2965]
 [  0.     661.2009 357.7372]
 [  0.       0.       1.    ]] 
[[659.5987   0.     643.7851]
 [  0.     659.888  365.5354]
 [  0.       0.       1.    ]] 
[[650.5334   0.     652.9999]
 [  0.     651.7737 371.0067]
 [  0.       0.       1.    ]] 
[[660.8861   0.     640.8387]
 [  0.     663.289  353.6643]
 [  0.       0.       1.    ]] 
[[653.5935   0.     659.2435]
 [  0.     653.262  370.4768]
 [  0.       0.       1.    ]]


In [7]:
path_stereo = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration\stereo'
path_intrinsics = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration\intrinsics_refine'

cal.find_matching_images(path, 'u101', cams, time_threshold, path_stereo, path_intrinsics)
cal.find_matching_images(path, 'u102', cams, time_threshold, path_stereo, path_intrinsics)

## Stereo calibration with matching images for each pair of cameras

### Loop on possible camera pairs

In [13]:
for camL in cams:
    for camR in cams[cams.index(camL)+1:]:
        print(camL, camR)
        stereopath = os.path.join(r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration\stereo', f'{camL}_{camR}')
        Left_Paths = [os.path.join(stereopath, fname) for fname in list(os.listdir(stereopath)) if fname.split('_')[2] == camL]
        Right_Paths = [os.path.join(stereopath, fname) for fname in list(os.listdir(stereopath)) if fname.split('_')[2] == camR]

        """ # TODO: verif images still match between Left_Paths and Right_Paths
        Left_Paths_conf, Right_Paths_conf = [], []
        for lpath, rpath in zip (Left_Paths, Right_Paths):
            if lpath[:5] == rpath[:5]:
                Left_Paths_conf.append(lpath)
                Right_Paths_conf.append(rpath)
            else:
                continue

        print('Before:', len(Left_Paths), len(Right_Paths))
        print('After:', len(Left_Paths_conf), len(Right_Paths_conf))
        # Verif OK """
        
        if len(Left_Paths) > 4 and len(Right_Paths) > 4:
            Left_imgpoints, Left_Paths, Right_imgpoints, Right_Paths = cal.GenerateImagepoints(Left_Paths, Right_Paths, CHECKERBOARD)
            if len(Left_imgpoints) > 4 and len(Right_imgpoints) > 4:

                try:
                    Ki = K[cams.index(camL)]
                    Di = D[cams.index(camL)]
                    Left_Params = cal.CalibrateCamera(Left_Paths, Left_imgpoints, K[cams.index(camL)], D[cams.index(camL)])

                except IndexError:
                    Left_Params = cal.CalibrateCamera(Left_Paths, Left_imgpoints)

                try:
                    Kj = K[cams.index(camR)]
                    Dj = D[cams.index(camR)]
                    Right_Params = cal.CalibrateCamera(Right_Paths, Right_imgpoints, Kj, Dj)
                except IndexError:
                    Right_Params = cal.CalibrateCamera(Right_Paths, Right_imgpoints)

                #Left_MeanError = cal.CalculateErrors(Left_Params, Left_imgpoints)
                #Right_MeanError = cal.CalculateErrors(Right_Params, Right_imgpoints)

                Left_MeanError = np.mean(np.array(Left_Params['Errors']), axis=-1)
                Right_MeanError = np.mean(np.array(Right_Params['Errors']), axis=-1)

                Left_Params['Imgpoints'] = Left_imgpoints
                #Left_Params['Errors'] = Left_Errors
                Left_Params['MeanError'] = Left_MeanError

                Right_Params['Imgpoints'] = Right_imgpoints
                #Right_Params['Errors'] = Right_Errors
                Right_Params['MeanError'] = Right_MeanError

                print('Reprojection Error Left:  {:.4f}'.format(Left_MeanError))
                print('Reprojection Error Right: {:.4f}'.format(Right_MeanError))                

                Stereo_Params = cal.StereoCalibration(Left_Params, Right_Params, Left_imgpoints, Right_imgpoints, Left_Paths)
                print('Transformation Matrix:')
                print(Stereo_Params['Transformation'])

                cal.SaveParameters(camL, camR, Stereo_Params, Left_Params, Right_Params)
            else:
                print('Not enough images', '\n')
        else:
            print('Not enough images', '\n')

c1 c2
Not enough images 

c1 c3
Not enough images 

c1 c4
Reprojection Error Left:  0.9345
Reprojection Error Right: 2.2651
Transformation Matrix:
[[   0.2646    0.3419   -0.9017 1160.4597]
 [  -0.257     0.9262    0.2758 -129.1217]
 [   0.9295    0.1587    0.333   274.4224]
 [   0.        0.        0.        1.    ]]
c1 c5
Not enough images 

c1 c6
Reprojection Error Left:  0.9552
Reprojection Error Right: 1.6249
Transformation Matrix:
[[   0.8471   -0.2547   -0.4664 -122.2723]
 [   0.1575    0.9585   -0.2375  278.9536]
 [   0.5076    0.1277    0.8521  230.6635]
 [   0.        0.        0.        1.    ]]
c2 c3
Reprojection Error Left:  1.6226
Reprojection Error Right: 1.8640
Transformation Matrix:
[[  0.7394   0.2675  -0.6179 631.1026]
 [ -0.2504   0.9611   0.1165   2.8659]
 [  0.625    0.0686   0.7776 512.1361]
 [  0.       0.       0.       1.    ]]
c2 c4
Not enough images 

c2 c5
Reprojection Error Left:  1.1058
Reprojection Error Right: 2.0268
Transformation Matrix:
[[   0.5564  

In [5]:
# c1_c2 = np.load('c1_c2_parameters.npz')['Transformation']
# c1_c3 = np.load('c1_c3_parameters.npz')['Transformation']
c1_c4 = np.load('c1_c4_parameters.npz')['Transformation']
# c1_c5 = np.load('c1_c5_parameters.npz')['Transformation']
c1_c6 = np.load('c1_c6_parameters.npz')['Transformation']
c2_c3 = np.load('c2_c3_parameters.npz')['Transformation']
c2_c5 = np.load('c2_c5_parameters.npz')['Transformation']
c2_c6 = np.load('c2_c6_parameters.npz')['Transformation']
# c3_c4 = np.load('c3_c4_parameters.npz')['Transformation']
c3_c5 = np.load('c3_c5_parameters.npz')['Transformation']
# c3_c6 = np.load('c3_c6_parameters.npz')['Transformation']
c4_c6 = np.load('c4_c6_parameters.npz')['Transformation']
c5_c6 = np.load('c5_c6_parameters.npz')['Transformation']

In [10]:
def var_name(var, scope=globals()):
    return [name for name, value in scope.items() if value is var][0]

def compare_transfo(T12, T23, T13):
    print(f'Original {var_name(T13)}')
    print(T13)
    T13_calc = T12 @ T23
    print(f'Calculated {var_name(T13)}')
    print(T13_calc)
    print('Calculated I')
    print(np.linalg.inv(T13_calc) @ T13, '\n')

    return

In [15]:
compare_transfo(np.linalg.inv(c1_c4), c1_c6, c4_c6)

Original c4_c6
[[    0.8273    -0.5613    -0.0234 -1144.6129]
 [    0.5576     0.8255    -0.0877   -89.054 ]
 [    0.0686     0.0595     0.9959  1157.0212]
 [    0.         0.         0.         1.    ]]
Calculated c4_c6
[[   0.6555   -0.195     0.7296 -484.9933]
 [   0.5161    0.821    -0.2442  -67.58  ]
 [  -0.5514    0.5366    0.6388 1254.6139]
 [   0.        0.        0.        1.    ]]
Calculated I
[[   0.7922    0.0252   -0.6097 -389.6578]
 [   0.3333    0.8191    0.467    58.6219]
 [   0.5112   -0.5731    0.6405 -538.3436]
 [   0.        0.        0.        1.    ]] 



In [16]:
compare_transfo(c2_c5, np.linalg.inv(c3_c5), c2_c3)

# compare_transfo(c2_c6, np.linalg.inv(c3_c6), c2_c3)

Original c2_c3
[[  0.7394   0.2675  -0.6179 631.1026]
 [ -0.2504   0.9611   0.1165   2.8659]
 [  0.625    0.0686   0.7776 512.1361]
 [  0.       0.       0.       1.    ]]
Calculated c2_c3
[[  0.824    0.1819  -0.5366 392.774 ]
 [ -0.2967   0.9453  -0.1353 108.4086]
 [  0.4826   0.2707   0.8329 611.8972]
 [  0.       0.       0.       1.    ]]
Calculated I
[[   0.9852   -0.0316   -0.1684  179.5587]
 [   0.0669    0.9758    0.2082  -83.4338]
 [   0.1578   -0.2164    0.9635 -196.6965]
 [   0.        0.        0.        1.    ]] 



In [17]:
# compare_transfo(np.linalg.inv(c1_c5), c1_c6, c5_c6)

compare_transfo(np.linalg.inv(c2_c5), c2_c6, c5_c6)

Original c5_c6
[[  -0.7047    0.6283   -0.3296 -461.7503]
 [  -0.6934   -0.5118    0.5071 -662.7922]
 [   0.1499    0.5859    0.7964  269.5371]
 [   0.        0.        0.        1.    ]]
Calculated c5_c6
[[    0.6997     0.2321     0.6757 -1614.4295]
 [   -0.2971    -0.7656     0.5706  -273.5667]
 [    0.6498    -0.6       -0.4667  1706.4582]
 [    0.         0.         0.         1.    ]]
Calculated I
[[  -0.1897    0.9724    0.1362  -11.51  ]
 [   0.2773    0.1861   -0.9426 1427.7082]
 [  -0.9419   -0.141    -0.305  1227.365 ]
 [   0.        0.        0.        1.    ]] 



## Intrinsics visualization

In [19]:
# List to store all intrinsic matrices
intrinsics = {}
for c in cams:
    intrinsics[c] = []

# Iterate over all .npz files in the folder
for cL in cams:
    for cR in cams[cams.index(cL)+1:]:
        file = f'{cL}_{cR}_parameters.npz'
        if file in os.listdir(os.getcwd()):
            data = np.load(file)
            intrinsics[cL].append(data['L_Intrinsic'])
            intrinsics[cR].append(data['R_Intrinsic'])

# Plot mean intrinsic matrix
plt.figure(figsize=(16, 8))
m, n = 1, 2

# Set fixed colorbar range (adjust these values based on your data)
mean_vmin, mean_vmax = 0, 700
std_vmin, std_vmax = 0, 70 # Std dev should start from 0

# Convert list to NumPy array (shape: num_files x 3 x 3)
for cam, K in intrinsics.items():
    K = np.array(K)

    # Compute mean and standard deviation
    mean_intrinsic = np.mean(K, axis=0)
    std_intrinsic = np.std(K, axis=0)

    # Print results
    print(f'\nCamera: {cam}')
    print("Mean Intrinsic Matrix:\n", mean_intrinsic)
    print("Standard Deviation of Intrinsic Matrix:\n", std_intrinsic)

    plt.subplot(3, 4, m)
    sns.heatmap(mean_intrinsic, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, vmin=mean_vmin, vmax=mean_vmax)
    plt.title(f"Mean Intrinsic Matrix for {cam}")

    # Plot standard deviation as uncertainty
    plt.subplot(3, 4, n)
    sns.heatmap(std_intrinsic, annot=True, fmt=".2f", cmap="coolwarm", cbar=True, vmin=std_vmin, vmax=std_vmax)
    plt.title(f"Uncertainty (Standard Deviation) for {cam}")

    plt.tight_layout()
    plt.axis('off')

    m += 2
    n += 2

    """ # Extract mean intrinsic parameters
    fx, fy = mean_intrinsic[0, 0], mean_intrinsic[1, 1]  # Focal lengths
    cx, cy = mean_intrinsic[0, 2], mean_intrinsic[1, 2]  # Principal point
    img_w, img_h = 1280, 720  # Example image resolution

    # Extract uncertainty (standard deviation)
    fx_std, fy_std = std_intrinsic[0, 0], std_intrinsic[1, 1]
    cx_std, cy_std = std_intrinsic[0, 2], std_intrinsic[1, 2]

    # === 1️⃣ Principal Point Visualization ===
    plt.figure(figsize=(8, 6))
    plt.scatter(cx, cy, color="red", label="Mean Principal Point", s=100)
    plt.errorbar(cx, cy, xerr=cx_std, yerr=cy_std, fmt='o', color='red', capsize=5, label="Uncertainty")
    plt.xlim(0, img_w)
    plt.ylim(img_h, 0)  # Invert y-axis to match image coordinates
    plt.axhline(img_h / 2, linestyle="--", color="gray", alpha=0.5)
    plt.axvline(img_w / 2, linestyle="--", color="gray", alpha=0.5)
    plt.title("Principal Point and Uncertainty")
    plt.xlabel("Image Width (pixels)")
    plt.ylabel("Image Height (pixels)")
    plt.legend()
    plt.grid()
    plt.show() """


plt.savefig('Intrinsics_mean_std.png')
plt.show()



Camera: c1
Mean Intrinsic Matrix:
 [[653.7606   0.     659.3578]
 [  0.     653.3662 370.8082]
 [  0.       0.       1.    ]]
Standard Deviation of Intrinsic Matrix:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Camera: c2
Mean Intrinsic Matrix:
 [[678.0557   0.     708.3392]
 [  0.     625.1823 366.6214]
 [  0.       0.       1.    ]]
Standard Deviation of Intrinsic Matrix:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Camera: c3
Mean Intrinsic Matrix:
 [[658.5619   0.     653.5985]
 [  0.     636.8952 373.7058]
 [  0.       0.       1.    ]]
Standard Deviation of Intrinsic Matrix:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Camera: c4
Mean Intrinsic Matrix:
 [[653.7606   0.     659.3578]
 [  0.     653.3662 370.8082]
 [  0.       0.       1.    ]]
Standard Deviation of Intrinsic Matrix:
 [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]

Camera: c5
Mean Intrinsic Matrix:
 [[673.9372   0.     642.6444]
 [  0.     646.5698 363.0538]
 [  0.       0.       1.    ]]
Standard Deviation of Intrinsic Matrix:
 [[ 4.7694  0

## Particle Swarm Optimizer pour paramètres intrinsèques

In [ ]:
#TODO: pso_optimize, bundle adjustment multi-camera (>2)

# autres méthodes d'optmisation -> PSO est le plus adapté à cette situation
# SolvePnP -> déjà utilisé

# erreur reprojection : détection, triangulation, reprojection images -> erreur
# tests sur nb images nécessaires/suffisantes pour calib

# (+ coverage BB/image) -> fait


par image et par caméra

In [6]:
path = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration\intrinsics\c1"

objpoints = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objpoints[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objpoints[:, :2] = objpoints[:, 0:2] * SQUARE_SIZE

im_name = os.path.join(path, os.listdir(path)[30])
img = cv2.imread(os.path.join(path, im_name))
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
_, imgpoints = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
imgpoints = imgpoints.squeeze()
imgpoints = imgpoints.astype(np.float32)

_, r, t = cv2.solvePnP(objpoints, imgpoints, K[0], D[0])
r = r.flatten()
t = t.flatten()

# Define the initial parameter values
params_c1 = K[0]
init_params = [params_c1[0,0], params_c1[1,1], params_c1[0,2], params_c1[1,2]]
init_params += list(D[0])
init_params += [r[0], r[1], r[2]]
init_params += [t[0], t[1], t[2]]

init_params = np.array(init_params, dtype=np.float32)  # Convert to NumPy array
print("Initial Parameters:", init_params)

# Define Optimizer hyperparameters
n_particles = 50  # Number of particles

init_pos = np.tile(init_params, (n_particles, 1))  # Shape: (30, 15)
print("Initial Position Shape:", init_pos.shape)  # Should be (30, 15)

param_bounds = ([500, 500, 600, 300, -0.1, -0.1, -0.01, -0.01, -0.1, -np.pi, -np.pi, -np.pi, -1000, -1000, -1000],  # Lower bounds
                [700, 700, 700, 400, 0.1, 0.1, 0.01, 0.01, 0.1, np.pi, np.pi, np.pi, 1000, 1000, 2000])          # Upper bounds
param_bounds = np.array(param_bounds, dtype=np.float32).reshape((2,15))
print(param_bounds)
print('Param bounds Shape:', param_bounds.shape)

options = {'c1': 1.5, 'c2': 0.3, 'w': 0.5}  # Cognitive, social, and inertia weights

# Define projection Model
def project_points(params, obj_pts):
    """
    Projects 3D points to 2D using intrinsic parameters.
    params = [fx, fy, cx, cy, k1, k2, p1, p2, k3] (9 parameters)
    """
    fx, fy, cx, cy, k1, k2, p1, p2, k3, r1, r2, r3, t1, t2, t3 = params
    
    camera_matrix = np.array([[fx, 0, cx],
                              [0, fy, cy],
                              [0,  0,  1] ], dtype=np.float32)
    dist_coeffs = np.array([k1, k2, p1, p2, k3])

    rvec = np.array([r1, r2, r3])
    tvec = np.array([t1, t2, t3])
    
    # Project points
    projected_pts, _ = cv2.projectPoints(obj_pts, rvec, tvec, camera_matrix, dist_coeffs)
    return projected_pts.squeeze()

# Define cost function
def RMSE(params):
    errors = np.zeros(n_particles)  # Array to store errors

    for i in range(n_particles):  
        imgp_proj = project_points(params[i], objpoints)  # Project points for this particle
        
        squared_diff = (imgpoints - imgp_proj) ** 2
        mse = np.mean(np.sum(squared_diff, axis=1))
        rmse = np.sqrt(mse)
        errors[i] = np.mean(rmse)  # Store mean RMSE for this particle
    
    return errors  # Return array of errors (one per particle)

# Run PSO to optimize calibration parameters
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=15, options=options, bounds=param_bounds, init_pos=init_pos)
best_error, best_params = optimizer.optimize(RMSE, iters=500)

plot_cost_history(cost_history=optimizer.cost_history)
plt.show()

# Extract optimized parameters
fx, fy, cx, cy, k1, k2, p1, p2, k3, r1, r2, r3, t1, t2, t3 = best_params

# Create final camera matrix
optimized_camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
optimized_dist_coeffs = np.array([k1, k2, p1, p2, k3])
optimized_rvec = np.array([r1, r2, r3])
optimized_tvec = np.array([t1, t2, t3])

print("Optimized Camera Matrix:\n", optimized_camera_matrix)
print("Optimized Distortion Coefficients:\n", optimized_dist_coeffs)
print("Optimized Rotation Vector:\n", optimized_rvec)
print("Optimized Translation Vector:\n", optimized_tvec)
print(f"Best Reprojection Error: {best_error}")


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (15,) + inhomogeneous part.

Pour toutes les images de toutes les caméras en même temps, (r,t) avec SolvePnP pour chaque image

In [ ]:
# Constants
num_cameras = 6
num_params_per_cam = 9
total_params = num_cameras * num_params_per_cam
n_particles = 50  # Number of particles
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.5}  # PSO Hyperparameters

# Paths to calibration images for each camera
base_path = r"C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration\intrinsics_refine"
camera_paths = [os.path.join(base_path, f'c{i+1}') for i in range(num_cameras)]

# Generate 3D object points (same for all images)
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2) * SQUARE_SIZE

# Initial parameters for each camera
K_fab = [[[644, 0, 656], [0, 643, 358], [0,0,1]],
         [[638, 0, 645], [0, 637, 363], [0,0,1]],
         [[645, 0, 645], [0, 643, 362], [0,0,1]],
         [[644, 0, 637], [0, 643, 358], [0,0,1]],
         [[642, 0, 650], [0, 641, 357], [0,0,1]],
         [[644, 0, 650], [0, 642, 362], [0,0,1]]]
K_fab = np.array(K_fab, dtype=np.float32)

K_calc = np.load('Intrinsics_K.npz')['arr_0']
D_calc = np.load('Intrinsics_D.npz')['arr_0']

K = K_fab       # Choose between theorical or calculated params K_fab / K_calc
D = [np.zeros(5, dtype=np.float32) for _ in range(num_cameras)]  # Distortion Coeffs

# Store object points and image points for all cameras
objpoints_list = []  # 3D points (same for all cameras)
imgpoints_list = [[] for _ in range(num_cameras)]  # 2D points per camera
extrinsics_list = [[] for _ in range(num_cameras)] 

for cam_idx, cam_path in enumerate(camera_paths):
    img_files = os.listdir(cam_path)
    for img_name in img_files:
        img_path = os.path.join(cam_path, img_name)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)
        if ret:
            corners = corners.squeeze().astype(np.float32)
            imgpoints_list[cam_idx].append(corners)
            objpoints_list.append(objp)

init_params = []
for cam_idx in range(num_cameras):
    cam_init = [K[cam_idx,0, 0], K[cam_idx,1, 1], K[cam_idx,0, 2], K[cam_idx,1, 2]]
    cam_init += list(D[cam_idx])
    init_params.extend(cam_init)

init_params = np.array(init_params, dtype=np.float32)

# Define parameter bounds
lower_bounds = [500, 500, 600, 300, -0.1, -0.1, -0.01, -0.01, -0.1]
upper_bounds = [700, 700, 700, 400, 0.1, 0.1, 0.01, 0.01, 0.1]

param_bounds = (np.tile(lower_bounds, num_cameras), np.tile(upper_bounds, num_cameras))

In [78]:
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.5}  # PSO Hyperparameters

# Projection function
def project_points(camera_matrix, dist_coeffs, r, t, obj_pts):
    # fx, fy, cx, cy, k1, k2, p1, p2, k3, r1, r2, r3, t1, t2, t3 = cam_params
    # camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0,  0,  1]], dtype=np.float32)
    # dist_coeffs = np.array([k1, k2, p1, p2, k3], dtype=np.float32)
    # rvec = np.array([r1, r2, r3])
    # tvec = np.array([t1, t2, t3])
    projected_pts, _ = cv2.projectPoints(obj_pts, r, t, camera_matrix, dist_coeffs)
    return projected_pts.squeeze()

# Define cost function
def RMSE(params):
    errors = np.zeros(n_particles)
    num_images = len(objpoints_list)  # Total images used for calibration
    
    for i in range(n_particles):
        total_error = 0
        for cam_idx in range(num_cameras):
            cam_params = params[i][cam_idx * num_params_per_cam : (cam_idx + 1) * num_params_per_cam]
            fx, fy, cx, cy, k1, k2, p1, p2, k3 = cam_params
            camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0,  0,  1]], dtype=np.float64)
            dist_coeffs = np.array([k1, k2, p1, p2, k3], dtype=np.float64)

            for (obj_pts, img_pts) in zip(objpoints_list, imgpoints_list[cam_idx]):
                _, r, t = cv2.solvePnP(obj_pts, img_pts, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_EPNP)

                projected_pts = project_points(camera_matrix, dist_coeffs, r, t, obj_pts)

                squared_diff = (img_pts - projected_pts) ** 2
                mse = np.mean(np.sum(squared_diff, axis=1))
                rmse = np.sqrt(mse)
                total_error += rmse
        
        errors[i] = total_error / (num_cameras * num_images)
    
    return errors[i]

# Run PSO to optimize calibration parameters
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=total_params, options=options, bounds=param_bounds)
best_error, best_params = optimizer.optimize(RMSE, iters=20)

# Extract optimized parameters for each camera
optimized_params = np.split(best_params, num_cameras)
for cam_idx in range(num_cameras):
    fx, fy, cx, cy, k1, k2, p1, p2, k3 = optimized_params[cam_idx]
    optimized_camera_matrix = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    optimized_dist_coeffs = np.array([k1, k2, p1, p2, k3])
    print(f"Camera {cam_idx+1} Optimized Parameters:")
    print("Optimized Camera Matrix:\n", optimized_camera_matrix)
    print("Optimized Distortion Coefficients:\n", optimized_dist_coeffs)
    print("\n")

print(f"Best Reprojection Error: {best_error}")

plot_cost_history(cost_history=optimizer.cost_history)
plt.show()


2025-02-28 09:42:44,303 - pyswarms.single.global_best - INFO - Optimize for 20 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.5}
pyswarms.single.global_best:  10%|█         |2/20, best_cost=1.46C:\Users\LEA\AppData\Local\Temp\ipykernel_76732\2601850050.py:31: RuntimeWarning: overflow encountered in square
  squared_diff = (img_pts - projected_pts) ** 2
pyswarms.single.global_best: 100%|██████████|20/20, best_cost=1.34
2025-02-28 09:44:36,865 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 1.3413528294807702, best pos: [629.0066 692.2739 609.1485 360.4797   0.0982  -0.0352   0.0028  -0.0043
  -0.0398 628.8125 615.7793 620.3899 378.6241  -0.0332  -0.0843   0.0002
  -0.0009  -0.0363 626.9632 587.0663 642.742  354.7231  -0.0028  -0.0474
  -0.0046   0.0006   0.024  578.8816 560.0724 619.7809 314.7627  -0.0416
   0.0479   0.0067   0.0004  -0.012  658.3184 525.7467 655.8684 359.267
   0.0647   0.0153   0.0063   0.0053   0.0202 512.6711 629.2437 674.0313
 306.5824  -0.0234 

Camera 1 Optimized Parameters:
Optimized Camera Matrix:
 [[629.0066   0.     609.1485]
 [  0.     692.2739 360.4797]
 [  0.       0.       1.    ]]
Optimized Distortion Coefficients:
 [ 0.0982 -0.0352  0.0028 -0.0043 -0.0398]


Camera 2 Optimized Parameters:
Optimized Camera Matrix:
 [[628.8125   0.     620.3899]
 [  0.     615.7793 378.6241]
 [  0.       0.       1.    ]]
Optimized Distortion Coefficients:
 [-0.0332 -0.0843  0.0002 -0.0009 -0.0363]


Camera 3 Optimized Parameters:
Optimized Camera Matrix:
 [[626.9632   0.     642.742 ]
 [  0.     587.0663 354.7231]
 [  0.       0.       1.    ]]
Optimized Distortion Coefficients:
 [-0.0028 -0.0474 -0.0046  0.0006  0.024 ]


Camera 4 Optimized Parameters:
Optimized Camera Matrix:
 [[578.8816   0.     619.7809]
 [  0.     560.0724 314.7627]
 [  0.       0.       1.    ]]
Optimized Distortion Coefficients:
 [-0.0416  0.0479  0.0067  0.0004 -0.012 ]


Camera 5 Optimized Parameters:
Optimized Camera Matrix:
 [[658.3184   0.     655.8684]
 

## 3D reprojection error

In [7]:
# Function to project 3D points to 2D
def project_points(points_3d, projMat, K, D):
    rvec, _ = cv2.Rodrigues(projMat[0:3,0:3])
    projected_2d, _ = cv2.projectPoints(points_3d, rvec, projMat[0:3,3], K, D)

    return projected_2d.squeeze()

# Compute RMSE
def compute_rmse(original_pts, points_3d, projMat, K, D):
    projected_pts = project_points(points_3d, projMat, K, D)
    
    error = np.linalg.norm(original_pts - projected_pts, axis=1)  # Euclidean distance per point
    rmse = np.sqrt(np.mean(error**2))  # Compute RMSE
    return rmse


## Stereo initial parameters

In [8]:
# Create array of all stereo images with 'im_name, img_pts, cam_ timestamp'
path_user = u101
user = 'u101'
stereo_images = {'Camera':[], 'Img_pts':[], 'Time':[]}

dir_stereo = r'C:\Users\LEA\Desktop\Poly\Trampo\data_cobotique\calibration\stereo'

for cam1 in cams:
    cali1 = cal.retrieve_cali_info(path, user, cam1[-1])
    for cam2 in cams[cams.index(cam1)+1:]:
        cali2 = cal.retrieve_cali_info(path, user, cam2[-1])
        path_stereo = os.path.join(dir_stereo, f'{cam1}_{cam2}')

        for i,t1 in enumerate(cali1[1,:]):
            t1 = float(t1)
            for j,t2 in enumerate(cali2[1,:]):
                t2 = float(t2)
                if abs(t1-t2) < time_threshold:
                    
                    im1In = f'{i}_{j}_{cam1}_{cali1[0,i]}' in list(os.listdir(path_stereo))
                    im2In = f'{i}_{j}_{cam2}_{cali2[0,j]}' in list(os.listdir(path_stereo))
                            
                    if im1In and im2In:

                        Lret, Limg, Limg_pts = cal.findCorners(os.path.join(path_stereo, f'{i}_{j}_{cam1}_{cali1[0,i]}'))
                        Rret, Rimg, Rimg_pts = cal.findCorners(os.path.join(path_stereo, f'{i}_{j}_{cam2}_{cali2[0,j]}'))

                        if Lret and Rret:
                            stereo_images['Camera'].append(int(cam1[-1]))
                            stereo_images['Img_pts'].append(Limg_pts)
                            stereo_images['Time'].append(t1)

                            stereo_images['Camera'].append(int(cam2[-1]))
                            stereo_images['Img_pts'].append(Rimg_pts)
                            stereo_images['Time'].append(t2)

with open('stereo_data.pkl', 'wb') as f:
    pickle.dump(stereo_images, f)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\LEA\\Desktop\\Poly\\Trampo\\data_cobotique\\calibration\\intrinsics\\c1\\cali1.csv'

In [9]:
with open('stereo_data.pkl', 'rb') as f:
    stereo_data = pickle.load(f)


In [10]:
# cam1 = world ref frame
# trouver ref frame de chaque cam dans world ref frame
# list of images, cam, timestamp (for all stereo images)
# triangulate to obtain 3d points
# project 3d points in each cam frame and compute RMSE
# optmise cam positions in world frame
num_cameras = 6

tworld = np.zeros((3,1)) # = cam1
Rworld = np.eye((3))
Tcam1 = np.hstack((Rworld, tworld))

tcam4 = c1_c4[0:3,3].reshape((3,1))
Rcam4 = c1_c4[0:3,0:3]
Tcam4 = np.hstack((Rcam4, tcam4))

tcam6 = c1_c6[0:3,3].reshape((3,1))
Rcam6 = c1_c6[0:3,0:3]
Tcam6 = np.hstack((Rcam6, tcam6))

c1_c5 = c1_c6 @ np.linalg.inv(c5_c6)
tcam5 = c1_c5[0:3,3].reshape((3,1))
Rcam5 = c1_c5[0:3,0:3]
Tcam5 = np.hstack((Rcam5, tcam5))

c1_c2 = c1_c6 @ np.linalg.inv(c2_c6)
tcam2 = c1_c2[0:3,3].reshape((3,1))
Rcam2 = c1_c2[0:3,0:3]
Tcam2 = np.hstack((Rcam2, tcam2))

c1_c3 = c1_c2 @ c2_c3
tcam3 = c1_c3[0:3,3].reshape((3,1))
Rcam3 = c1_c3[0:3,0:3]
Tcam3 = np.hstack((Rcam3, tcam3))

projMat = [Tcam1, Tcam2, Tcam3, Tcam4, Tcam5, Tcam6]

K_opt = []
D_opt = []

""" for cam_idx in range(num_cameras):
    fx, fy, cx, cy, k1, k2, p1, p2, k3 = K[cam_idx]
    K_opt.append(np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]]))
    D_opt.append(np.array([k1, k2, p1, p2, k3])) """

for mat in projMat:
    print(mat)


[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[   -0.6815    -0.2958     0.6693 -1484.8568]
 [    0.2409    -0.9544    -0.1766   629.7269]
 [    0.691      0.0409     0.7217  1387.8244]]
[[   -0.0114    -0.4207     0.9071 -1573.0064]
 [    0.3067    -0.8649    -0.3973   688.5534]
 [    0.9517     0.2737     0.1389  2193.6478]]
[[   0.2646    0.3419   -0.9017 1160.4597]
 [  -0.257     0.9262    0.2758 -129.1217]
 [   0.9295    0.1587    0.333   274.4224]]
[[  -0.6033   -0.6936   -0.3937 -754.4096]
 [   0.5695   -0.7202    0.3961  -42.2131]
 [  -0.5583    0.0147    0.8295 -240.9416]]
[[   0.8471   -0.2547   -0.4664 -122.2723]
 [   0.1575    0.9585   -0.2375  278.9536]
 [   0.5076    0.1277    0.8521  230.6635]]


## PSO for extrinsic parameters (with Bundle adjustment)
### Inspired from https://scipy-cookbook.readthedocs.io/items/bundle_adjustment.html

In [11]:
# Constants
num_cameras = 6
num_params_per_cam = 6
total_params = num_cameras * num_params_per_cam
n_particles = 60  # Number of particles
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.7}  # PSO Hyperparameters

init_params = []
for mat in projMat: #[1:]:
    rvec, _ = cv2.Rodrigues(mat[0:3,0:3])
    rvec = rvec.squeeze()
    params = [rvec[0], rvec[1], rvec[2], mat[0,3], mat[1,3], mat[2,3]]
    init_params.extend(params)

init_params = np.array(init_params, dtype=np.float64)
init_pos = np.tile(init_params, (n_particles,1))
print(init_params.shape)
print(init_pos.shape)

# Define parameter bounds
lower_bounds = [-2*np.pi, -2*np.pi, -2*np.pi, -3000, -3000, -3000]
upper_bounds = [2*np.pi, 2*np.pi, 2*np.pi, 3000, 3000, 3000]

param_bounds = (np.tile(lower_bounds, num_cameras), np.tile(upper_bounds, num_cameras))
print(param_bounds[0].shape)

print(np.all(param_bounds[0] <= init_pos[0]))
print(np.all(init_pos[0] <= param_bounds[1]))

(36,)
(60, 36)
(36,)
True
True


In [35]:
def fun(params):
    errors = np.empty((n_particles,))
    params = np.array(params)

    for n in range(n_particles):
        projMat = np.empty((num_cameras, 3, 4))
        #projMat[0] = np.hstack((np.eye(3), np.zeros((3,1)))) # Camera 1 as reference

        for cam_idx in range(num_cameras):
            cam_params = params[n][cam_idx * num_params_per_cam : (cam_idx + 1) * num_params_per_cam]
            r1, r2, r3, t1, t2, t3 = cam_params
            rvec = np.array([r1, r2, r3])
            R, _ = cv2.Rodrigues(rvec)
            t = np.array([t1, t2, t3]).reshape((3,1))
            projMat[cam_idx] = np.hstack((R, t))  #[cam_idx+1]

        RMSE = []
        for i in range(0, len(stereo_data['Camera']) - 1, 2):
            j = i+1 # stereo image is the next one

            pts1_im = stereo_data['Img_pts'][i].squeeze()
            pts2_im = stereo_data['Img_pts'][j].squeeze()

            c1 = int(stereo_data['Camera'][i]) - 1
            c2 = int(stereo_data['Camera'][j]) - 1

            t1 = float(stereo_data['Time'][i])
            t2 = float(stereo_data['Time'][j])
            assert abs(t1-t2) < time_threshold, f'Images are not matching: {t1:.4f} and {t2:.4f}'

            undist_pts1 = cv2.undistortPoints(pts1_im, K[c1], D[c1]).reshape(-1, 2).T  # Shape (2, N)
            undist_pts2 = cv2.undistortPoints(pts2_im, K[c2], D[c2]).reshape(-1, 2).T  # Shape (2, N)

            # Perform triangulation
            pts_4d = cv2.triangulatePoints(projMat[c1], projMat[c2], undist_pts1, undist_pts2)

            # Convert from homogeneous coordinates
            points_3d = pts_4d[:3] / pts_4d[3]  # Shape (3, N)
            points_3d = points_3d.T  # Shape (N, 3)


            # Compute RMSE for both cameras
            rmse1 = compute_rmse(pts1_im, points_3d, projMat[c1], K[c1], D[c1])
            rmse2 = compute_rmse(pts2_im, points_3d, projMat[c2], K[c2], D[c2])

            RMSE.append(rmse1)
            RMSE.append(rmse2)

        RMSE = np.array(RMSE, dtype=np.float64)
        #RMSE[RMSE > 1000] = np.nan

        errors[n] = np.nanmean(RMSE)

    return errors

""" #_, RMSE = fun(stereo_data)
plt.figure()
plt.plot(range(len(RMSE)), RMSE)
plt.xlabel('Im index')
plt.ylabel('3D Reprojection RMSE (pixels)')
plt.show()
plt.close()

print(np.nanmean(RMSE), np.nanmedian(RMSE)) """

""" print(f"RMSE Camera {i+1}: {rmse1:.4f} pixels")
print(f"RMSE Camera {j+1}: {rmse2:.4f} pixels")

plt.scatter(pts1_im[:,0], pts1_im[:,1], c='r', label="Original")
plt.scatter(projected_pts1[:,0], projected_pts1[:,1], c='b', label="Projected")
plt.legend()
plt.show() """


' print(f"RMSE Camera {i+1}: {rmse1:.4f} pixels")\nprint(f"RMSE Camera {j+1}: {rmse2:.4f} pixels")\n\nplt.scatter(pts1_im[:,0], pts1_im[:,1], c=\'r\', label="Original")\nplt.scatter(projected_pts1[:,0], projected_pts1[:,1], c=\'b\', label="Projected")\nplt.legend()\nplt.show() '

In [36]:
# Run PSO to optimize calibration parameters
optimizer = ps.single.GlobalBestPSO(n_particles=n_particles, dimensions=total_params, options=options, bounds=param_bounds, init_pos=init_pos)
best_error, best_params = optimizer.optimize(fun, iters=300)

# Extract optimized parameters for each camera
optimized_params = np.split(best_params, num_cameras)

extrinsics = []

for cam_idx in range(num_cameras):
    r1, r2, r3, t1, t2, t3 = optimized_params[cam_idx]
    rvec = np.array([r1,r2,r3], dtype=np.float64)
    R, _ = cv2.Rodrigues(rvec)
    t = np.array([t1,t2,t3], dtype=np.float64).reshape((3,1))
    optimized_camera_matrix = np.hstack((R,t))

    extrinsics.append(optimized_camera_matrix)
    print(f"Camera {cam_idx+1} Optimized Parameters:")
    print("Optimized Camera Matrix:\n", optimized_camera_matrix)
    print("\n")

print(f"Best Reprojection Error: {best_error} pixels")

np.savez('Extrinsics_optimized_Kman.npz', np.round(np.array(extrinsics), decimals=4))

plot_cost_history(cost_history=optimizer.cost_history)
plt.savefig('Figure_1_pso_cost_Kman.png')
plt.show()



2025-03-05 11:19:39,389 - pyswarms.single.global_best - INFO - Optimize for 300 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.7}
pyswarms.single.global_best:   0%|          |1/300, best_cost=6.24e+7c:\Users\LEA\Desktop\Poly\Trampo\code\trampo\Lib\site-packages\numpy\linalg\linalg.py:2582: RuntimeWarning: overflow encountered in multiply
  s = (x.conj() * x).real
pyswarms.single.global_best:  67%|██████▋   |201/300, best_cost=33.8 c:\Users\LEA\Desktop\Poly\Trampo\code\trampo\Lib\site-packages\numpy\linalg\linalg.py:2583: RuntimeWarning: overflow encountered in reduce
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))
pyswarms.single.global_best: 100%|██████████|300/300, best_cost=29  
2025-03-05 11:26:53,098 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 29.01800314016771, best pos: [    0.0456     0.3014     0.3225     4.6229    -1.3611    65.0911
     1.2258     0.2396     3.1663 -1494.7199   -55.4905   146.9503
     2.0845     0.0354     2.346  -1570.936

Camera 1 Optimized Parameters:
Optimized Camera Matrix:
 [[ 0.9042 -0.3053  0.2988  4.6229]
 [ 0.3188  0.9478  0.0037 -1.3611]
 [-0.2843  0.0919  0.9543 65.0911]]


Camera 2 Optimized Parameters:
Optimized Camera Matrix:
 [[   -0.7109     0.2909     0.6403 -1494.7199]
 [   -0.1912    -0.9561     0.222    -55.4905]
 [    0.6768     0.0354     0.7353   146.9503]]


Camera 3 Optimized Parameters:
Optimized Camera Matrix:
 [[   -0.1178     0.0127     0.993  -1570.9369]
 [    0.0173    -0.9997     0.0148   771.833 ]
 [    0.9929     0.0189     0.1175  1915.0872]]


Camera 4 Optimized Parameters:
Optimized Camera Matrix:
 [[   0.7358    0.16     -0.658  1160.1616]
 [  -0.087     0.986     0.1425 -116.716 ]
 [   0.6716   -0.0476    0.7394  220.7148]]


Camera 5 Optimized Parameters:
Optimized Camera Matrix:
 [[  -0.964     0.2125    0.16   -758.6053]
 [  -0.2221   -0.974    -0.0445  -75.9075]
 [   0.1464   -0.0785    0.9861 -228.8847]]


Camera 6 Optimized Parameters:
Optimized Camera Matrix:

## Cumulative error on matrices multiplication (Total transfo = I) 

In [51]:
T = []
extrinsics = np.load('Extrinsics_optimized_Kman.npz')['arr_0']
for mat in extrinsics:
    T.append(np.vstack((mat, np.array([0,0,0,1]))))

T_next = []
for i in range(len(T) - 1):
    T_next.append(np.linalg.inv(T[i]) @ T[i+1])

print('T5 calc: \n', T_next[0] @ T_next[1] @ T_next[2] @ T_next[3] @ T_next[4])
print('T5: \n', T[5])

I_calc = T_next[0] @ T_next[1] @ T_next[2] @ T_next[3] @ T_next[4] @ np.linalg.inv(T[5])
print('Cumulative error: \n', np.abs(I_calc - np.eye(4)))


T5 calc: 
 [[  0.8521  -0.1313  -0.5066 -67.8455]
 [  0.1573   0.9875   0.0084 353.3117]
 [  0.4991  -0.0869   0.8622 119.9638]
 [  0.       0.       0.       1.    ]]
T5: 
 [[   0.8716   -0.4462   -0.203  -128.7439]
 [   0.4226    0.8938   -0.1503  312.3224]
 [   0.2485    0.0452    0.9676  231.3304]
 [   0.        0.        0.        1.    ]]
Cumulative error: 
 [[ 0.0959  0.3188  0.2843 14.7617]
 [ 0.3053  0.0522  0.0919  3.2813]
 [ 0.2988  0.0037  0.0457 63.4945]
 [ 0.      0.      0.      0.    ]]


In [31]:
def plot_frame(ax, T, label="Frame", length=100.0):
    """Plots a 3D reference frame with arrows for X, Y, and Z axes."""
    origin = T[:3, 3]  # Extract translation (position)
    R = T[:3, :3]      # Extract rotation matrix (orientation)

    # Define the x, y, z unit vectors from the rotation matrix
    x_axis = R[:, 0] * length  # X-direction
    y_axis = R[:, 1] * length  # Y-direction
    z_axis = R[:, 2] * length  # Z-direction

    # Plot arrows for the axes
    ax.quiver(*origin, *x_axis, color='r', arrow_length_ratio=0.3)  # X-axis (Red)
    ax.quiver(*origin, *y_axis, color='g', arrow_length_ratio=0.3)  # Y-axis (Green)
    ax.quiver(*origin, *z_axis, color='b', arrow_length_ratio=0.3)  # Z-axis (Blue)

    # Label the frame
    ax.text(*origin, label, fontsize=12, color='black')

    return

In [45]:
extrinsics = np.load('Extrinsics_optimized.npz')['arr_0']

# Plot frames
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i, T in enumerate(extrinsics):
    T = np.vstack((T, np.array([0,0,0,1])))
    plot_frame(ax, T, label=f'Camera {i+1}')

# Labels
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
plt.title("3D Reference Frames")

plt.show()
